In [5]:
from flask import Flask
from flask_restful import Resource, Api, reqparse
import pandas as pd
import ast
import psycopg2
import pandas.io.sql as sqlio
import requests
import sqlalchemy
from sqlalchemy import create_engine
from pathlib import Path
import json
from multipledispatch import dispatch
import os

In [6]:
def get_properties():
    props = {}
    separator = "="
            
    with open('./database.properties') as file:
        for line in file: 
            if separator in line:
                name, value = line.split(separator, 1)
                props[name.strip()] = value.strip()
    return props

### Setup database

In [7]:
class Database:
    def __init__(self):
        db_prop = get_properties()
        self.username = db_prop.get('username')
        self.password = db_prop.get('password')
        self.host = db_prop.get('host')
        self.port = db_prop.get('port')
        self.database = db_prop.get('database')
        #For SQLAlchemy
       
    
    def connect(self):
        connection = psycopg2.connect(user=self.username,
                                      password=self.password,
                                      host=self.host,
                                      port=self.port,
                                      database=self.database)
        return connection, connection.cursor()
    
    def connect_alchemy(self):
        '''Alt library to connect used for importing dataframes'''
        engine = create_engine('postgresql://'+self.username+':'+self.password+'@'+self.host+':'+self.port+'/'+self.database)
        return engine
    
    @dispatch(str)
    def insert_habit(self, name):
        query = 'INSERT INTO public.task ("name") VALUES (%s);'
        values = tuple(name)
        conn, cursor = None, None
        try:
            conn, cursor = self.connect()
            cursor.execute(query, (name,))
            conn.commit()
        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
        finally:
            # closing database connection.
            if conn:
                cursor.close()
                conn.close()
    @dispatch(str, str)            
    def insert_habit(self, name, date):
        query = 'INSERT INTO public.task ("name", "date") VALUES (%s, %s);'
        values = (name, date,)
        conn, cursor, task_id = None, None, None
        try:
            conn, cursor = self.connect()
            cursor.execute(query, values)
            conn.commit()
            ## BROKEN
            task_id = cursor.fetchone()[0]
        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
        finally:
            # closing database connection.
            if conn:
                cursor.close()
                conn.close()
            return task_id
        
    def insert_dataframe(self, df):
        conn = None
        try:
            conn = self.connect_alchemy().connect()
            df.to_sql('task', con=conn, if_exists='append', index=False)
        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
        finally:
            # closing database connection.
            if conn:
                conn.close()
                
    def insert_exercise(self, exercise):
        query = """INSERT INTO public.exercise ("average_heart_rate", "calories", "duration", "steps", "distance", "task_fk", "timestamp") 
        VALUES  (%s,%s, %s, %s, %s, %s, %s);"""
        conn, cursor = None, None
        try:
            conn, cursor = self.connect()
            task_fk = sqlio.read_sql_query("select id from task where name = '"+exercise['name']+"' and date = '"+exercise['date']+"'", conn)
            if task_fk.empty:
                task_fk = self.insert_habit(exercise['name'], exercise['date'])
            
            values = (exercise['avg_heart'],  exercise['calories'],  exercise['duration'],  exercise['steps'], exercise['distance'], int(task_fk['id'][0]), exercise['time'])
            cursor.execute(query, values)
            conn.commit()
        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
        finally:
            # closing database connection.
            if conn:
                cursor.close()
                conn.close()
    def insert_meditation(self, meditation):
        query = """ INSERT into public.meditation ("type","duration", task_fk) VALUES (%s, %s, %s)"""
        con, cursor = None, None
        try:
            conn, cursor = self.connect()
            task_fk = sqlio.read_sql_query("select id from task where name = 'Meditate' and date = '"+meditation['date']+"'", conn)
            if task_fk.empty:
                task_fk = self.insert_habit('Meditate', meditation['date'])
            values = (meditation['type'], meditation['duration'], int(task_fk['id'][0]))
            cursor.execute(query, values)
            conn.commit()
        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
        finally:
            # closing database connection.
            if conn:
                cursor.close()
                conn.close()
        
    def read_habits(self):
        query = "select * from task"
        result = None
        conn, cursor = None, None
        try:
            conn, cursor = self.connect()
            result = sqlio.read_sql_query(query, conn)
        except(Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
        finally:
            if conn:
                cursor.close()
                conn.close()
            return result         

In [8]:
def db_conn_test():
    db = Database()
    db.insert_habit("test231")
    tables = db.read_habits()
    return tables
#db_conn_test()

### Habitica Import

In [9]:
class Habitica():
    def __init__(self):
        props = get_properties()
        dataExport = "https://habitica.com/api/v3/tasks/user"
        self.headers = {'x-client': props.get('x-client'),
              'x-api-user': props.get('x-api-user'),
              'x-api-key': props.get('x-api-key')}
        request = requests.get(dataExport, headers = self.headers)
        self.task_list = request.json()['data']
        
    def get_tasks(self):
        ''' Get a list of all the tasks I'm tracking '''
        tasks = [{'Name':task['text'], 'id':task['id']} for task in self.task_list if '#' not in task['text'] and 'Reward' not in task['text'] and task['type'] in 'daily' or 'habit']
        cleaned = filter(lambda x: '##' not in x['Name'], tasks)
        cleaned = filter(lambda x: 'Reward' not in x['Name'], cleaned)
        return list(cleaned)
    
    def get_history(self):
        '''Get a dataframe of the days I finished any tasks'''
        task_hist_api = "https://habitica.com/api/v3/tasks/"
        task_hist = []
        #Get a list of the tasks we are tracking
        tasks = self.get_tasks()
        #Create a list of dataframes we will append to
        df_list = [pd.DataFrame({'date', 'name'})]
        # For each task loop through and grab the history of that task, 
        # then pull out only the relevent info for that task IE: did we do it that day
        # Merge all the dataframes together and clean 
        for task in tasks:
            response = requests.get(task_hist_api+task['id'], headers=self.headers).json()
            df = pd.DataFrame(response['data']['history'])
            df['name'] = task['Name']
            rows_list = []
            #df = df.dropna()
            if df.columns[3] == 'completed':
                ## Dailies
                for index, row in df.iterrows():
                    if index == 0 and row['value'] > 1:
                        rows_list.append({'name': row['name'], 'date':pd.to_datetime(row['date'],unit='ms')})
                    elif row['value'] > df.iloc[index - 1].value:
                        rows_list.append({'name': row['name'], 'date':pd.to_datetime(row['date'],unit='ms')})
            else:
                #Habits
                for index, row in df.iterrows():
                    if row['scoredUp'] + row['scoredDown'] > 0:
                        rows_list.append({'name': row['name'], 'date':pd.to_datetime(row['date'],unit='ms')})
            #Create dataframe
            df_list.append(pd.DataFrame(rows_list))
        #Clean data
        df_final = pd.concat(df_list)
        df_final = df_final.drop([0], axis = 1)
        df_final = df_final.sort_values(by = ['date'])
        df_final['date'] = df_final['date'].dt.strftime('%Y-%m-%d')
        df_final = df_final.reset_index()
        df_final = df_final.drop(['index'], axis = 1)
        df_final = df_final.dropna()
        df_final = df_final[df_final.duplicated() == False]
        return df_final
    
    def refresh_database(self):
        db = Database()
        tasks = self.get_history()
        for row in tasks.iterrows():
            db.insert_habit(row[1]["name"],row[1]["date"])
        return tasks
        

habitica = Habitica()
tasks =habitica.refresh_database()
#habitica.get_history()

Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL duplicate key value violates unique constraint "task_un"
DETAIL:  Key (name, date)=(Walk Dog, 2021-01-04) already exists.

Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL duplicate key value violates unique constraint "task_un"
DETAIL:  Key (name, date)=(Meditate, 2021-01-04) already exists.

Error while connecting to PostgreSQL duplicate key value violates unique constraint "task_un"
DETAIL:  Key (name, date)=(Clean Patrick, 2021-01-04) already exists.

Error while connecting to PostgreSQL duplicate key value violates unique constraint "task_un"
DETAIL:  Key (name, date)=(Programming, 2021-01-04) already exists.

Error while connecting to PostgreSQL duplicate key value violates unique constraint "task_un"
DETAIL:  Key (name, date)=(Cat

In [13]:
df = habitica.get_history()

### Fit Import

In [10]:
class Fit():
    def __init__(self):
        props = get_properties()
        self.directory = Path(props['fit_file_location'])
    
    def read_exercise(self):
        exercise_path = self.directory.joinpath('Physical Activity')
        files = []
        exercises = []
        for entry in os.scandir(exercise_path):
            if("exercise" in str(entry)):
                files.append(entry)  
        for file in files:
            with open(file) as exercise_file:
                data = json.load(exercise_file)
                exercises_data = [self.__load_run(exercise) for exercise in data if exercise['activityName'] in 'Run']
                exercises_data = exercises_data + [self.__load_other(exercise) for exercise in data if exercise['activityName'] in 'Yoga']
                exercises_data = exercises_data + [self.__load_other(exercise) for exercise in data if exercise['activityName'] in 'Weights']
                exercises = exercises+exercises_data
        return exercises
    
    def refresh_database(self):
        db = Database()
        exercises = self.read_exercise()
        for exercise in exercises:
            db.insert_exercise(exercise)
        
    
    def __load_run(self, run):
        run_simp = {"name":"Run", "time": pd.to_datetime(run['startTime']).strftime('%H:%M:%S'), "date": pd.to_datetime(run['startTime']).strftime('%Y-%m-%d') , "avg_heart": run['averageHeartRate'], "calories":run['calories'], "duration": run['duration'], "steps": run['steps']}
        if('distance' in run):
            run_simp.update({"distance" : run['distance']})
        else:
            run_simp.update({"distance" : 0})
        return run_simp
    
    def __load_other(self, other):
        other_simp = {"name":other["activityName"], "time": pd.to_datetime(other['startTime']).strftime('%H:%M:%S'), "date": pd.to_datetime(other['startTime']).strftime('%Y-%m-%d') , "calories":other['calories'], 
                      "duration": other['duration'], "avg_heart": other['averageHeartRate'], "steps": 0, "distance":0}
        return other_simp
        
    
    
fit = Fit()
fit.read_exercise()
fit.refresh_database()


KeyboardInterrupt: 

### Waking up Import

In [16]:
class WakingUp():
    def __init__(self):
        props = get_properties()
        self.directory = Path(props['wakingup_file_location'])
        
    def read_wakingup_file(self):
        exercise_path = self.directory.joinpath('wu_progress.csv')
        meditation_history = pd.read_csv(exercise_path, header=0, names=["date", "type", "duration"])
        meditation_history["date"] = pd.to_datetime(meditation_history.date).dt.strftime('%Y-%m-%d') 
        return meditation_history
    
    def refresh_database(self):
        db = Database()
        meditations = self.read_wakingup_file()
        for index, meditation in meditations.iterrows():
            db.insert_meditation(meditation)
        

waking = WakingUp()
#df = waking.read_wakingup_file()
waking.refresh_database()
#df['date']
        
    

### Transfer Pre-2020 data

In [73]:
class Old_Database:
    def __init__(self):
        db_prop = get_properties()
        self.username = db_prop.get('username')
        self.password = db_prop.get('password')
        self.host = db_prop.get('host')
        self.port = db_prop.get('port')
        self.database = "Habits"
        #For SQLAlchemy
       
    
    def connect(self):
        connection = psycopg2.connect(user=self.username,
                                      password=self.password,
                                      host=self.host,
                                      port=self.port,
                                      database=self.database)
        return connection, connection.cursor()
    
    def read_habits(self):
        query = "select activity.date, category.name from activity inner join category ON activity.category_fk = category.id"
        result = None
        conn, cursor = None, None
        try:
            conn, cursor = self.connect()
            result = sqlio.read_sql_query(query, conn)
        except(Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
        finally:
            if conn:
                cursor.close()
                conn.close()
            result["name"] = result["name"].str.title()
            return result
    
    def migrate_data(self, db):
        old_habits = self.read_habits()
        for row in old_habits.iterrows():
            db.insert_habit(row[1]["name"], str(row[1]["date"]))
        
        
    

In [74]:
old_db = Old_Database()

In [75]:
old_db.migrate_data(Database())

Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to PostgreSQL no results to fetch
Error while connecting to Postg

In [76]:
df

,date,name
0,2019-06-19,Yoga
1,2019-09-18,Yoga
2,2019-10-08,Yoga
3,2019-10-15,Yoga
4,2019-10-26,Yoga
...,...,...
794,2020-10-30,Walk Dog
795,2020-10-29,Running
796,2020-10-29,Yoga
797,2020-10-29,Meditation


In [65]:
for row in df.iterrows():
    print(row[1]["name"])

Yoga
Yoga
Yoga
Yoga
Yoga
Yoga
Yoga
Yoga
Weights
Programming
Walk Dog
Reading
Meditation
Programming
Programming
Meditation
Meditation
Meditation
Meditation
Walk Dog
Programming
Meditation
Programming
Programming
Programming
Meditation
Meditation
Meditation
Meditation
Meditation
Meditation
Yoga
Running
Running
Running
Running
Running
Running
Running
Running
Running
Running
Running
Yoga
Yoga
Meditation
Running
Meditation
Meditation
Walk Dog
Walk Dog
Yoga
Reading
Reading
Programming
Reading
Meditation
Yoga
Meditation
Yoga
Yoga
Yoga
Meditation
Yoga
Meditation
Yoga
Meditation
Meditation
Meditation
Yoga
Meditation
Yoga
Meditation
Meditation
Running
Running
Running
Meditation
Running
Meditation
Running
Meditation
Meditation
Meditation
Reading
Meditation
Walk Dog
Reading
Walk Dog
Meditation
Walk Dog
Meditation
Programming
Reading
Programming
Running
Yoga
Yoga
Yoga
Yoga
Yoga
Yoga
Yoga
Running
Programming
Yoga
Programming
Meditation
Weights
Meditation
Programming
Programming
Meditation
Programmi